In [1]:
import numpy as np

In [123]:
# SET THIS TO LOAD EXAMPLE DATA INSTEAD
# example = True
example = False

if example:
    fname = 'example.txt'
else:
    fname = 'input.txt'

with open(fname) as f:
    lines = f.readlines()

heightmap = [[int(char) for char in line.strip()] for line in lines]
hm = np.array(heightmap)

In [124]:
%%time

r, c = hm.shape
shifted_right = np.hstack([np.full((r,1), 9), hm[:,0:-1]])
shifted_left = np.hstack([hm[:,1:], np.full((r,1), 9)])
shifted_up = np.vstack([hm[1:,:], np.full((1,c), 9)])
shifted_down = np.vstack([np.full((1,c), 9), hm[0:-1,:]])

low_mask = (hm < shifted_right) & (hm < shifted_left) & (hm < shifted_down) & (hm < shifted_up)
sum_risk = (hm[low_mask] + 1).sum()
print(f"sum of risk levels: {sum_risk}")

sum of risk levels: 537
Wall time: 0 ns


In [125]:
a = set([0, 1])
b = set([1, 2])
# c = set(a).update(set(b))


In [126]:
def count_neighbors(r, c, r_max, c_max, already_checked, heightmap):
    """ # depth-first search to find all neighbors of higher, but not 9, height """
    
    already_checked.add((r, c))

    if heightmap[r, c] < 9:

        # propose new neighbors
        potential_neighbors = []
        for r_p, c_p in [(r - 1, c), (r, c - 1), (r + 1, c), (r, c + 1)]:
            if (0 <= r_p < r_max) and \
            (0 <= c_p < c_max) and \
            (r_p, c_p) not in already_checked:
                potential_neighbors.append((r_p, c_p))

#         print(f"current: {(r, c)}\t potential: {potential_neighbors}")
        # check new neigbors
        neighbor_set = set()
        for pn in potential_neighbors:
            if pn not in already_checked:
                neighbors, a_c = count_neighbors(*pn, r_max, c_max, already_checked, heightmap)
                already_checked.update(a_c)
                neighbor_set.update(neighbors)

        # add current point
        neighbor_set.add((r, c))
        return neighbor_set, already_checked
    
    else:
        return set(), already_checked

In [131]:
def count_neighbors(r, c, r_max, c_max, already_checked, heightmap):
    """ # depth-first search to find all neighbors of higher, but not 9, height """
    
    already_checked.add((r, c))

    if heightmap[r, c] < 9:

        # propose new neighbors
        potential_neighbors = []
        for r_p, c_p in [(r - 1, c), (r, c - 1), (r + 1, c), (r, c + 1)]:
            if (0 <= r_p < r_max) \
            and (0 <= c_p < c_max):
                potential_neighbors.append((r_p, c_p))

        # check new neigbors
        neighbor_set = set()
        for pn in potential_neighbors:
            if pn not in already_checked: # already_checked may be updated between iterations 
                neighbor_set.update(count_neighbors(*pn, r_max, c_max, already_checked, heightmap))

        # add current point
        neighbor_set.add((r, c))
        return neighbor_set
    
    else:
        return set()

In [132]:
%%time

basin_sizes = []
for coord in np.argwhere(low_mask):
    basin_sizes.append(len(count_neighbors(*coord, *hm.shape, set(), hm)))
prod = np.prod(sorted(basin_sizes)[-3:])

print(f"product of 3 largest: {prod}")


product of 3 largest: 1142757
Wall time: 60 ms


In [59]:
# 1142757